# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


**Описание данных**
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Обзор данных

In [1]:
# импортируйте библиотеку pandas
import pandas as pd

In [2]:
# прочитайте csv-файл
data = pd.read_csv('data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
mediana = data.groupby('income_type')['total_income'].median()
for income_t in data['income_type'].unique():
    data.loc[data['income_type'] == income_t, 'total_income'] = data['total_income'].fillna(mediana[income_t])

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально.

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Однако данные не понадобяться для исследования, поэтому оставим как есть.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения.

In [10]:
data = (
    data
    .loc[(data['children'] != -1) & (data['children'] != 20)]
    .reset_index(drop=True)
)

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [12]:
mediana = data.groupby('income_type')['days_employed'].median()
for income_t in data['income_type'].unique():
    data.loc[data['income_type'] == income_t, 'days_employed'] = data['days_employed'].fillna(mediana[income_t])

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропуски заполнены.

### Изменение типов данных

In [14]:
data['total_income'] = data['total_income'].astype('int') 

### Обработка дубликатов

В столбце `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

Создадим столбец `total_income_category` в датафрейме `data`  с категориями на основании следующих диапозонов:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [18]:
def categorize_income(income):
    if 0 <= income <= 30000:
        return 'E'
    elif income <= 50000:
        return 'D'
    elif income <= 200000:
        return 'C'
    elif income <= 1000000:
        return 'B'
    return 'A'

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Создим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose or 'недвижим' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
# Сгруппируем данные по числу детей у заемщиков и посмотрим на число долгов в каждой группе
data_grouped_children = data.groupby('children').agg({'debt': ['count', 'sum']})

# Рассчитаем долю имеющих задолженность
data_grouped_children['ratio_debt'] = data_grouped_children['debt']['sum'] / data_grouped_children['debt']['count']
data_grouped_children

debt       ratio_debt
          count   sum           
children                        
0         14091  1063   0.075438
1          4808   444   0.092346
2          2052   194   0.094542
3           330    27   0.081818
4            41     4   0.097561
5             9     0   0.000000

**Вывод:** Из таблицы `data_grouped_children` видно, что как при отсутсвии, так и при наличии детей у заемщиков бывают задолженности. Причем от числа детей наличие долга тоже не зависит: у людей с одним, двумя и четырьмя детьми доля задолженностей почти одинакова.

Хотя число задолженностей у людей с 5 детьми нет, данная выборка слишком мала относительно другим, чтобы делать на этом вывод.

Сравним только наиболее многочисленные группы из таблицы: без детей, с 1 ребенком и с 2 детьми. Тогда заметно, что задолженность среди заемщиков без детей меньше, чем у тех, что с детьми (особенно если учесть, что выборка при этом больше).

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
# Сгруппируем данные по семейному положению заемщиков и посмотрим на число долгов в каждой группе
data_grouped_family = data.groupby('family_status').agg({'debt': ['count', 'sum']})

# Рассчитаем долю имеющих задолженность
data_grouped_family['ratio_debt'] = data_grouped_family['debt']['sum'] / data_grouped_family['debt']['count']
data_grouped_family

debt      ratio_debt
                       count  sum           
family_status                               
Не женат / не замужем   2796  273   0.097639
в разводе               1189   84   0.070648
вдовец / вдова           951   63   0.066246
гражданский брак        4134  385   0.093130
женат / замужем        12261  927   0.075606

**Вывод:** Из таблицы `data_grouped_family` видно, что у заемщиков с противоположным семейным положением женат/замужем - в разводе, не женат/не замужем - гражданский брак схожая доля задолженностей, пирчем первая пара имеет меньше задолженностей, чем вторая. Лучше всего справляются люди из категории вдовец/вдова, которые также являются самой малочисленной группой, поэтому делать выводы на этом ненадежно.

Если же сравнить только самые многочисленные категории: женат/замужем, гражданский брак, не женат/не замужем,видно, что заемщики состоящие в браке имеют меньше задолженностей.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
'''Сгруппируем данные по доходу заемщиков, где
A - получают больше миллиона в месяц,
B - получают от 200 тысяч до миллиона,
C - получают от 50 до 200 тысяч,
D - получают от 30 до 50 тысяч,
E - получают меньше 30 тысяч в месяц
и посмотрим на число долгов в каждой группе''' 
data_grouped_income = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})

# Рассчитаем долю имеющих задолженность
data_grouped_income['ratio_debt'] = data_grouped_income['debt']['sum'] / data_grouped_income['debt']['count']
data_grouped_income

debt       ratio_debt
                       count   sum           
total_income_category                        
A                         25     2   0.080000
B                       5014   354   0.070602
C                      15921  1353   0.084982
D                        349    21   0.060172
E                         22     2   0.090909

**Вывод:** из таблицы `data_grouped_income` видно, что хоть доля долгов немного отличается у людей с разным уровнем дохода, эта разница небольшая и нелинейная.
Но если сравнивать только самые большие группы В и С, то как и следовало ожидать, у людей с большим доходом меньшая доля задолженностей.

### Как разные цели кредита влияют на его возврат в срок?

In [26]:
# Сгруппируем данные по цели заемщиков и посмотрим на число долгов в каждой группе
data_grouped_purpose = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})

# Рассчитаем долю имеющих задолженность
data_grouped_purpose['ratio_debt'] = data_grouped_purpose['debt']['sum'] / data_grouped_purpose['debt']['count']
data_grouped_purpose

debt      ratio_debt
                          count  sum           
purpose_category                               
операции с автомобилем     4279  400   0.093480
операции с недвижимостью  10751  780   0.072551
получение образования      3988  369   0.092528
проведение свадьбы         2313  183   0.079118

**Вывод:**  из таблицы `data_grouped_purpose` видно, что доля задолженностей у желающих совершить операцию с автомобилем и получить образование больше, чем у других.

### Возможные причины появления пропусков в исходных данных.

**Ответ:** пропуски в представленном датафрейме могут быть связаны с отсутсвием необходимых данных у заемщика, например заработок или стаж у безработных и студентов, или данные по какой-то причине нельзя разглашать; пропуски могли появится по техническим причинам или в силу человеческой ошибки.

### Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных?

**Ответ:** пропуски в количественных переменных заполняют характерными значениями. Это значения, характеризующие состояние выборки, — набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки, годятся среднее арифметическое или медиана.

В данном случае подойдет именно медиана, потому что среднее значение может быть смещенно из-за очень большой или очень маленькой зарплаты у малочисленной части группы.

## Общий вывод

В процессе предобработки данных были выявлены аномальные данные, явные и неявные дубликаты и пропуски в данных. По возможности данные были скорректированы, но в процессе удаления некоторых строк могли потеряться важные для вывода данные. 

Мы проверили 4 гипотезы и установили:

1. Между количеством детей и возвратом кредита в срок нет явной зависимости. Однако заемщики без детей имеют меньшее число задолженностей.

Выводы могли быть другими, если бы в данных о количестве детей не было аномалий.

2. Между семейным положением и возвратом кредита в срок есть зависимость. Заемщики состоящие в браке имеют меньшее число задолженностей. Также мало задолженностей у разведенных и вдовствующих заемщиков, но их выборка значительно уступает остальным.

Вторая гипотеза подтвердилась.

3. Между уровнем дохода и возвратом кредита в срок нет явной зависимости. Однако среди самых многочисленных выборок прослеживается, что заемщики с большим доходом имеют меньше задолженностей.

Третья гипотеза подтвердилась отчасти. Результаты могли быть другими. если бы в данных не было пропусков.

4. Разные цели кредита влияют на его возврат в срок. Доля должников среди желающих совершить операцию с автомобилем и получить образование больше, чем у других.

Последняя гипотеза подтвердилась.